In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import contextily as ctx
import geopandas as gp
from geopy.geocoders import Nominatim
from collections import Counter
from fake_useragent import UserAgent
import json

In [14]:
all_df = pd.read_csv('pay-gaps-all-employers.csv')
ori_ldn_df = pd.read_csv('pay-gaps-london-employers.csv')

In [15]:
ori_ldn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2701 entries, 0 to 2700
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   EmployerName               2701 non-null   object
 1   Address                    2701 non-null   object
 2   CompanyNumber              2460 non-null   object
 3   SicCodes                   2522 non-null   object
 4   DiffMeanHourlyPercent      2701 non-null   object
 5   DiffMedianHourlyPercent    2701 non-null   object
 6   DiffMeanBonusPercent       2372 non-null   object
 7   DiffMedianBonusPercent     2372 non-null   object
 8   MaleBonusPercent           2701 non-null   object
 9   FemaleBonusPercent         2701 non-null   object
 10  MaleLowerQuartile          2701 non-null   object
 11  FemaleLowerQuartile        2701 non-null   object
 12  MaleLowerMiddleQuartile    2701 non-null   object
 13  FemaleLowerMiddleQuartile  2701 non-null   object
 14  MaleUppe

In [16]:
ori_ldn_df.head()

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline
0,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline
1,46 BERKELEY SQUARE LTD,"26-28, Conway Street,\r\nLondon,\r\nEngland,\r...",9497091,94990,3.1,2.1,NaN,NaN,0,0,...,34,74,26,78,22,NaN,Joanne Bolding (HR Manager),250 to 499,46 BERKELEY SQUARE LTD,FALSE
2,5 HERTFORD STREET LIMITED,"10 Norwich Street,\r\nLondon,\r\nEC4A 1BD",6913368,82990,-7.1,-14.2,15.8,-13.2,85,85,...,34,78,22,94,6,https://www.5hertfordstreet.com/home,Catherine Bristow (Human Resources Director),250 to 499,5 HERTFORD STREET LIMITED,FALSE
3,6Cats UK Ltd.,"1st Floor 1-3 Sun Street,\r\nLondon,\r\nEnglan...",7284675,70229,15,6,NaN,NaN,0,0,...,44,58,42,58,42,NaN,Kirsty Henning (Finance Director),250 to 499,6Cats UK Ltd.,FALSE
4,A & L RESTAURANTS LIMITED,"THE ACCOUNTING CENTRE, FIRST FLOOR,\r\n736 Hig...",4437266,56290,9.3,1,-44.3,88.9,2.6,3.6,...,65.2,45.1,54.9,54.6,45.4,NaN,Amir Atefi (Director),500 to 999,A & L RESTAURANTS LIMITED,FALSE


In [17]:
ldn_df = ori_ldn_df.iloc[1:,:].reset_index(drop=True)
ldn_df['postcode'] = ldn_df['Address'].apply(lambda x: x.split('\r\n')[-1])
ldn_df['sic'] = ldn_df['SicCodes'].apply(lambda x: str(x).split(',\r\n')[0][0:2])

In [18]:
ldn_df.head()

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,postcode,sic
0,46 BERKELEY SQUARE LTD,"26-28, Conway Street,\r\nLondon,\r\nEngland,\r...",9497091,94990,3.1,2.1,NaN,NaN,0,0,...,26,78,22,NaN,Joanne Bolding (HR Manager),250 to 499,46 BERKELEY SQUARE LTD,FALSE,W1T 6BQ,94
1,5 HERTFORD STREET LIMITED,"10 Norwich Street,\r\nLondon,\r\nEC4A 1BD",6913368,82990,-7.1,-14.2,15.8,-13.2,85,85,...,22,94,6,https://www.5hertfordstreet.com/home,Catherine Bristow (Human Resources Director),250 to 499,5 HERTFORD STREET LIMITED,FALSE,EC4A 1BD,82
2,6Cats UK Ltd.,"1st Floor 1-3 Sun Street,\r\nLondon,\r\nEnglan...",7284675,70229,15,6,NaN,NaN,0,0,...,42,58,42,NaN,Kirsty Henning (Finance Director),250 to 499,6Cats UK Ltd.,FALSE,EC2A 2EP,70
3,A & L RESTAURANTS LIMITED,"THE ACCOUNTING CENTRE, FIRST FLOOR,\r\n736 Hig...",4437266,56290,9.3,1,-44.3,88.9,2.6,3.6,...,54.9,54.6,45.4,NaN,Amir Atefi (Director),500 to 999,A & L RESTAURANTS LIMITED,FALSE,N12 9QD,56
4,A F A RESTAURANTS LIMITED,"THE ACCOUNTING CENTRE, FIRST FLOOR,\r\n736 Hig...",8345484,47290,1.7,0.2,12.8,8.2,6,5.7,...,49.3,62,38,NaN,Ahmet Mustafa (Director),500 to 999,A F A RESTAURANTS LIMITED,FALSE,N12 9QD,47


In [19]:
ldn_df.set_index('EmployerName',inplace=True)
employers = ldn_df.index
address = ldn_df.loc[:,'postcode'].to_list()

In [20]:
# ua = UserAgent()
# user_agent = ua.random
# geolocator = Nominatim(user_agent=user_agent)

# coords = {}
# for i,j in zip(employers,address):
#     try:
#         location = geolocator.geocode(j)
#         print(location)

#     except:
#         location = None
#         print("Location not found!")

#     if location is not None:
#         coords[i] = {
#             "Lat": location.latitude,
#             "Long": location.longitude
#         }
#     else:
#         pass

In [23]:
with open('coords.json', 'r') as f:
    coor_df = json.load(f)